## Loading The Data Packets

In [37]:
import warnings
import os
import socket
from scapy.all import *
import subprocess

warnings.filterwarnings("ignore")

packets = rdpcap("data-3-laptop/three_laptop.pcap")
MOB_IPv4 = "30.121.209.222"
MOB_IPv6 = "2607:fb90:a90e:af30:95e0:7380:4637:8872"

print(packets)

# Splitting up the packet types
tcp_packets = packets[TCP]
udp_packets = packets[UDP]
icmp_packets = packets[ICMP]

def IP2DomainName(IP):
    """
    This function converts the IP address to the domain name
    """
    dom_name = socket.gethostbyaddr(IP)[0]
    return dom_name

def DomainName2IP(url_name):
    """
    This function converts the domain name to an ip address
    """
    domain_ip = socket.gethostbyname(url_name)
    return domain_ip

<three_laptop.pcap: TCP:34972 UDP:3629 ICMP:89 Other:22379>


In [38]:
tcp_ipv4, tcp_ipv6 = [], []
for tcp_packet in tcp_packets:
    if IPv6 in tcp_packet:
        tcp_ipv6.append(tcp_packet)
    else:
        tcp_ipv4.append(tcp_packet)

print("IPv6 Packets: ", len(tcp_ipv6))
print("IPv4 Packets: ", len(tcp_ipv4))

IPv6 Packets:  5414
IPv4 Packets:  29558


## Identifying Local and Global Addresses (IPv4)

In [44]:
all_IPS, local_IPS = set(), set()

# List all the local IP addresses that starts with 192.168.*
for packet in packets:
    try:
        all_IPS.add(packet[IP].src)
        all_IPS.add(packet[IP].dst)
        if packet[IP].src.startswith("192.168"):
            local_IPS.add(packet[IP].src)
        if packet[IP].dst.startswith("192.168"):
            local_IPS.add(packet[IP].dst)
    except:
        pass

# List all the URL from the IP addresses
all_hosts = set()
for ip_addr in all_IPS:
    try:
        addr = socket.gethostbyaddr(ip_addr)[0]
        # print(ip_addr, "\t", addr)
        all_hosts.add((ip_addr, addr))
    except:
        pass

# Findout the Operating System of the Host
print("\n\n\n\n Local IP Addresses")
for local_IP in local_IPS:
    found = False
    if local_IP.endswith(".1"):
        print(f"{local_IP}: Android OS \t TTL(64)")
    for tcp_packet in tcp_packets:
        try:
            if tcp_packet[IP].src == local_IP:
                if tcp_packet[IP].ttl == 64:
                    print(f"{local_IP}: Unix Machine \t TTL(64) ")
                    found = True
                elif tcp_packet[IP].ttl==128:
                    print(f"{local_IP}: Windows Machine\t TTL(128) ")
                    found = True
            if found:
                break
        except:
            pass
    


print("\n\n\n Global IP Addresses")
print("See the full table at", "all_IPv4_url.txt file" )

with open("all_IPv4_url.txt", 'w') as fp:
    print(f"{'IP Address': <16}","\t", "URL", file=fp)
    for ip in all_IPS:
        try:
            print(f"{ip: <16}",'\t', IP2DomainName(ip), file=fp)
        except Exception as e:
            print(f"{ip: <16}",'\t', "<<   failed to resolve    >>", file=fp)

subprocess.call(["cat", "all_IPv4_url.txt"])







 Local IP Addresses
192.168.43.1: Android OS 	 TTL(64)
192.168.43.135: Unix Machine 	 TTL(64) 
192.168.43.230: Windows Machine	 TTL(128) 
192.168.43.136: Unix Machine 	 TTL(64) 



 Global IP Addresses
See the full table at all_IPv4_url.txt file
IP Address       	 URL
3.95.117.96      	 ec2-3-95-117-96.compute-1.amazonaws.com
104.87.85.175    	 a104-87-85-175.deploy.static.akamaitechnologies.com
18.160.10.38     	 server-18-160-10-38.iad12.r.cloudfront.net
172.253.122.95   	 bh-in-f95.1e100.net
45.60.149.216    	 <<   failed to resolve    >>
30.121.209.222   	 <<   failed to resolve    >>
44.240.84.205    	 ec2-44-240-84-205.us-west-2.compute.amazonaws.com
35.82.74.0       	 ec2-35-82-74-0.us-west-2.compute.amazonaws.com
173.223.187.178  	 a173-223-187-178.deploy.static.akamaitechnologies.com
142.251.163.95   	 wv-in-f95.1e100.net
23.212.249.82    	 a23-212-249-82.deploy.static.akamaitechnologies.com
142.250.31.93    	 bj-in-f93.1e100.net
20.110.81.91     	 <<   failed to resolve  

0

## Identifying Local and Global Addresses (IPv6)

In [69]:

global_IPv6 = set()
for packet in tcp_ipv6:
    try:
        if packet[IPv6].src != MOB_IPv6:
            global_IPv6.add(packet[IPv6].src)
        elif packet[IPv6].dst != MOB_IPv6:
            global_IPv6.add(packet[IPv6].dst)
    except:
        pass

print("\n\n\n Local IPv6 Addresses")
print(f"{MOB_IPv6}\n\n")
print("\n\n\n Global IPv6 Addresses")
print("See the full table at", "all_IPv6_url.txt file" )

with open("all_IPv6_url.txt", 'w') as fp:
    print(f"{'IP Address': <16}","\t", "URL", file=fp)
    for ip in global_IPv6:
        try:
            print(f"{ip: <16}",'\t', IP2DomainName(ip), file=fp)
        except:
            pass

subprocess.call(["cat", "all_IPv6_url.txt"])





 Local IPv6 Addresses
2607:fb90:a90e:af30:95e0:7380:4637:8872





 Global IPv6 Addresses
See the full table at all_IPv6_url.txt file
IP Address       	 URL
2607:f8b0:4004:c1b::5e 	 wv-in-f94.1e100.net
2607:f8b0:4004:c1b::8a 	 wv-in-f138.1e100.net
2607:f8b0:4004:810::2003 	 iad23s63-in-x03.1e100.net
2607:f8b0:4004:c07::5f 	 bc-in-f95.1e100.net
2607:f8b0:4004:c1b::54 	 wv-in-f84.1e100.net
2607:f8b0:4004:c1d::69 	 ww-in-f105.1e100.net
2607:f8b0:4004:c17::5f 	 bl-in-f95.1e100.net
2607:f018:1:1::1 	 www.umich.edu
2607:f8b0:4004:83e::200a 	 iad66s01-in-x0a.1e100.net
2607:f8b0:4004:c17::5e 	 bl-in-f94.1e100.net
2607:f8b0:4004:c1b::5f 	 wv-in-f95.1e100.net
2607:f8b0:4004:c1b::65 	 wv-in-f101.1e100.net
2607:f8b0:4004:83f::2003 	 iad23s04-in-x03.1e100.net
2607:f8b0:4004:c06::84 	 bg-in-f132.1e100.net
2607:f8b0:4004:c17::66 	 bl-in-f102.1e100.net
2607:f8b0:4004:c06::5f 	 bg-in-f95.1e100.net
2607:f8b0:4004:c17::9a 	 bl-in-f154.1e100.net
2607:f8b0:4004:801::200a 	 iad30s10-in-x0a.1e100.net
2607

0

### Filtering Out All Upstream Packets (IPv4):

In [59]:
# for IPv4
for packet in tcp_ipv4:
    if packet[IP].src == MOB_IPv4 and packet[IP].dst not in local_IPS:
        print(packet[IP].src, "\t", packet[IP].dst, "\t", packet[IP].ttl)

30.121.209.222 	 18.160.10.38 	 63
30.121.209.222 	 18.154.227.57 	 63
30.121.209.222 	 18.154.227.57 	 63
30.121.209.222 	 18.154.227.57 	 63
30.121.209.222 	 3.220.159.105 	 63
30.121.209.222 	 3.220.159.105 	 63
30.121.209.222 	 3.220.159.105 	 63
30.121.209.222 	 3.220.159.105 	 63
30.121.209.222 	 3.220.159.105 	 63
30.121.209.222 	 3.220.159.105 	 63
30.121.209.222 	 3.220.159.105 	 63
30.121.209.222 	 3.220.159.105 	 63
30.121.209.222 	 104.87.85.175 	 63
30.121.209.222 	 104.87.85.175 	 63
30.121.209.222 	 104.87.85.175 	 63
30.121.209.222 	 104.87.85.175 	 63
30.121.209.222 	 104.87.85.175 	 63
30.121.209.222 	 104.87.85.175 	 63
30.121.209.222 	 44.240.84.205 	 63
30.121.209.222 	 44.240.84.205 	 63
30.121.209.222 	 44.240.84.205 	 63
30.121.209.222 	 44.240.84.205 	 63
30.121.209.222 	 44.240.84.205 	 63
30.121.209.222 	 44.240.84.205 	 63
30.121.209.222 	 18.160.10.38 	 63
30.121.209.222 	 23.220.124.232 	 63
30.121.209.222 	 23.220.124.232 	 63
30.121.209.222 	 23.220.124.

In [67]:
# for IPv4
for packet in tcp_ipv6:
    if packet[IPv6].src == MOB_IPv6:
        try:
            print(packet[IPv6].src, "\t",
                    packet[IPv6].dst, "\t",
                    socket.gethostbyaddr(packet[IPv6].dst)[0], "\t", 
                    packet[IPv6].hlim)
        except:
            pass

2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c1b::5f 	 wv-in-f95.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c1b::5f 	 wv-in-f95.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c06::5e 	 bg-in-f94.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c06::5e 	 bg-in-f94.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c06::5e 	 bg-in-f94.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c06::5e 	 bg-in-f94.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c06::5e 	 bg-in-f94.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c06::5e 	 bg-in-f94.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c06::5e 	 bg-in-f94.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c06::5e 	 bg-in-f94.1e100.net 	 255
2607:fb90:a90e:af30:95e0:7380:4637:8872 	 2607:f8b0:4004:c1b::54 	 wv-

In [62]:
tcp_ipv6[0].show()


###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x212
  lladdrlen = 0
  src       = ''
  proto     = IPv6
###[ IPv6 ]### 
     version   = 6
     tc        = 0
     fl        = 0
     plen      = 32
     nh        = TCP
     hlim      = 255
     src       = 2607:fb90:a90e:af30:95e0:7380:4637:8872
     dst       = 2607:f8b0:4004:c1b::5f
###[ TCP ]### 
        sport     = 43655
        dport     = https
        seq       = 4079785978
        ack       = 2078829592
        dataofs   = 8
        reserved  = 0
        flags     = FA
        window    = 385
        chksum    = 0x4bcd
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (289847, 2666217924))]



In [34]:
upstream_packets, downstream_packets = [], []
destinations = []

for packet in packets:
    try:
        if packet[IP].src==MOB_IP and not packet[IP].dst.startswith("192.168"):
            upstream_packets.append(packet)
            destinations.append(packet[IP].dst)
    except: 
        pass
print("Number of upstream packets: ", len(upstream_packets))

print(list(set(destinations)))


ttls = set()
for packet in upstream_packets:
    try:
        print(packet[IP].src, "\t", packet[IP].ttl, "\t", packet[IP].dst, "\t", IP2DomainName(packet[IP].dst))
        ttls.add(packet[IP].ttl)
    except:
        pass

print(ttls)


Number of upstream packets:  1117
['17.253.21.204', '23.214.80.62', '17.253.21.206', '17.142.171.17', '35.9.41.236', '52.85.247.96', '17.248.169.83', '17.57.144.184', '157.240.249.61', '142.251.163.94', '142.251.33.195', '142.251.16.138', '34.239.39.161', '17.133.234.32', '104.96.203.25', '17.242.179.27', '172.253.62.94', '172.253.115.106', '17.36.206.8', '44.237.180.172', '142.251.16.100', '17.253.31.203', '172.253.122.84', '142.251.167.94', '52.204.91.203', '142.251.45.3', '10.177.0.34', '44.235.54.54', '44.234.235.93', '52.89.90.67', '54.235.48.238', '10.177.0.210', '172.253.63.101', '142.250.31.132', '35.83.181.24', '172.253.62.95', '54.163.175.227', '142.251.16.95', '172.253.122.95', '35.9.30.51', '172.253.63.188']
25.6.9.45 	 63 	 44.234.235.93 	 ec2-44-234-235-93.us-west-2.compute.amazonaws.com
25.6.9.45 	 63 	 35.83.181.24 	 ec2-35-83-181-24.us-west-2.compute.amazonaws.com
25.6.9.45 	 63 	 35.83.181.24 	 ec2-35-83-181-24.us-west-2.compute.amazonaws.com
25.6.9.45 	 63 	 35.83.18

## Looking at ICMP Packages

In [58]:
for icmp_packet in packets[ICMP]:
    try:
        print(f"{icmp_packet[IP].src}:{icmp_packet.sport} --> {icmp_packet[IP].dst}:{icmp_packet.dport}, TTL: {icmp_packet[IP].ttl}")
    except:
        pass

30.121.209.222:53 --> 10.177.0.210:43101, TTL: 64
30.121.209.222:53 --> 10.177.0.34:43101, TTL: 64
30.121.209.222:53 --> 10.177.0.210:43101, TTL: 64
30.121.209.222:53 --> 10.177.0.34:1970, TTL: 64
30.121.209.222:53 --> 10.177.0.210:1970, TTL: 64
192.168.43.135:53 --> 192.168.43.1:22984, TTL: 64
192.168.43.135:53 --> 192.168.43.1:32963, TTL: 64
192.168.43.135:53 --> 192.168.43.1:19220, TTL: 64
192.168.43.135:53 --> 192.168.43.1:16972, TTL: 64
192.168.43.135:53 --> 192.168.43.1:56357, TTL: 64
192.168.43.135:53 --> 192.168.43.1:55090, TTL: 64
192.168.43.135:53 --> 192.168.43.1:2290, TTL: 64
192.168.43.135:53 --> 192.168.43.1:18958, TTL: 64
192.168.43.135:53 --> 192.168.43.1:28279, TTL: 64
30.121.209.222:53 --> 10.177.0.210:16820, TTL: 64
30.121.209.222:53 --> 10.177.0.34:16820, TTL: 64
30.121.209.222:53 --> 10.177.0.210:42279, TTL: 64
30.121.209.222:53 --> 10.177.0.34:42279, TTL: 64
30.121.209.222:53 --> 10.177.0.210:42279, TTL: 64
30.121.209.222:53 --> 10.177.0.34:54208, TTL: 64
30.121.2

## Analyzing UDP Packets

In [17]:
udp_packets = packets[UDP]

In [32]:
for udp_packet in udp_packets:
    srcIP = udp_packet.sprintf("%IP.src%")
    dstIP = udp_packet.sprintf("%IP.dst%")
    srcPort = udp_packet.sprintf("%TCP.sport%")
    dstPort = udp_packet.sprintf("%TCP.dport%")
    
    ttl = None
    try:
        ttl = int(udp_packet.sprintf("%IP.ttl%"))
    except:
        ttl = udp_packet.sprintf("%IP.ttl%")

    try:
        print(f"Src: {srcIP}:{srcPort}  -->> Dest: {dstIP}:{dstPort}  {udp_packet.an.rrname} {ttl}")
    except:
        print(f"Src: {srcIP}:{srcPort}  -->> Dest: {dstIP}:{dstPort}  {ttl}")


Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  b'clientservices.googleapis.com.' ??
Src: ??:??  -->> Dest: ??:??  b'www.google.com.' ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  b'accounts.google.com.' ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  b'clientservices.googleapis.com.' ??
Src: ??:??  -->> Dest: ??:??  b'www.google.com.' ??
Src: ??:??  -->> Dest: ??:??  b'accounts.google.com.' ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest: ??:??  ??
Src: ??:??  -->> Dest

In [57]:
socket.gethostbyaddr('2a03:2880:f003:100:face:b00c:0:3')[0]

'xx-fbcdn6-shv-02-iad3.fbcdn.net'

In [73]:
packets[25000].show()

###[ cooked linux ]### 
  pkttype   = unicast
  lladdrtype= 0x212
  lladdrlen = 0
  src       = '\\x92B\\x8cq\\xf4\\xe9\\x89['
  proto     = 0xda1a
###[ Raw ]### 
     load      = '\x00\x01\x05\\x8cE\x00\x05\\x8c\\\\xff@\x00&\x06\\xed\x08gR\\xad\\xba\x1ey\\xd1\\xde\x01\\xbb\\xccX3\\xac`Yw\x1d\\x9fw\\x80\x10\x01\x08I\\x84\x00\x00\x01\x01\x08\n\x13\\xa9\\xf3\\x85\\xf2\\x95\x03\\xe1\x04\x7f-a\\x84G\\xcco\\xbba5S\\xa3\\xa1*\\xe70k\\xaa*\\xb9uYHeg\\x88\ueece\\x9e\x1b\\xd1\\xe9̔@\x11)\\x93\\x97\\xb73H=\\xd4Ę\t\x0c\\xc5\x06\\xea/\\xa9F\\xfft%?\x05o\\x9e\\xefǍ"\\x8f\x13\\xa6\\xf5\x19|n\\x91\\xb4cT\\x8fcK\\xba\x04kV\x03G\\xde/]\\xbc\\xcanJ\\xee\'!\\x9bi`\\x8c\\x85\\xf2\x14l=\\xebygG\\x95\\xefM\\x88;\x1c$w\\xd9cM\x1d;!%\\xe1\\x99\\xe2A;\\x86U\\xcf\\xeb\x06\\xfc\x08\\xb6c>ʹ3\\x94\x1d\\xb4\\x95\x0b\\x98u\\xd7V&\x1f\x11i\\x89I\\xd0+\\xd3\\xca\\xf5m+H\\xf3\\xdd3\\xaa\\xbb\\xc7c$\\x94£\\xfbB\\xf0#\\xd0\x15\\xc0M^\\xc0)\x1a\\xad\\xfc\\x85\\x96f\\x93\\xbe\\x8c\\xb7׃\\xf0p\x12\\xf4ִƨ\\xa4b\\xd8\\xf3|;\\